# RESUME DE NOTRE PROJET

Notre projet d'informatique porte sur l'implémentation d'un jeu d'échecs et de stratégies. 
    Avant de réfléchir à une quelconque stratégie, l'un d'entre nous s'est chargé de coder l'ensemble des règles du jeu d'échecs, en commençant par les règles de déplacements des pièces, puis en programmant le reste (comment vérifier si un joueur est en échec, comment caractériser l'échec et mat...). Nous avons également ajouté les règles moins essentielles du jeu, telles que la prise en passant, le roque et la promotion d'un pion.
    En parallèle, l'autre membre de notre binôme s'est occupé de l'interface graphique de notre projet. Nous sommes soucieux d'avoir un programme qui puisse être joué de manière assez simple, en cliquant sur une pièce puis sur la case sur laquelle on veut la déplacer. Il a donc fallu créer le visuel du plateau ainsi que des boutons cliquables sur chaque case de ce dernier.
    Enfin, nous nous sommes tous deux penchés sur une stratégie à adopter afin que l'utilisateur puisse jouer contre l'ordinateur. 

# Implémentation des règles de déplacements
## Notations et modules à importer

In [24]:
import copy
import tkinter

Dans chaque fonction de déplacement suivante, p correspondra à une liste de la forme [(X,Y), type, couleur, numero, image] où (X,Y) seront ses coordonnées sur le plateau. Image correspond à la représentation graphique de la pièce sur l'interface Le plateau est quant à lui représenté par une matrice. plateau[a,b] renvoit la pièce qui se trouve sur les coordonnées (a,b) du plateau. On caractérisera les cases vides par CV, définie par : [[0,0],"case_vide","",0,case_vide], où case_vide correspond à l'image de la case vide sur l'interface graphique. La variable morts en argument de chaque fonction correspond à la liste des pièces éliminées au cours de la partie. 

## Règles de déplacements

Tout d'abord, nous avons codé la fonction qui permet le déplacement d'un pion. Nous avons dû également prendre en compte dans cette fonction la possibilité de promouvoir un pion (le changer en une autre pièce lorsqu'il arrive de l'autre côté du plateau), ainsi que la prise en passant.

Pour implémenter la fonction promotion, il a fallu distinguer tous les cas de promotion du pion possible. Puis il est nécessaire de remplacer la pièce "morte" par le pion qui obtient sa promotion. Dès lors il était nécessaire d'accéder à l'indice de la liste "morts" dans lequel était stockée la pièce qui va remplacer le pion, d'où la création de la fonction indice.

In [25]:
def indice(x, liste) :               # renvoit l'indice de la liste dans lequel se trouve l'élément x
    indice = 0
    while liste[indice] != x and indice <= len(liste):
        indice += 1
    if indice == len(liste):
        return None
    return indice

In [4]:
# PROMOTION

In [5]:
# PRISE EN PASSANT

Finalement, la fonction deplacement_pion est assez dense, d'autant plus qu'il est nécessaire de distinguer la couleur du pion pour vérifier si le déplacement demandé est possible.

In [6]:
# DEPLACEMENT PION

Pour se déplacer avec la tour ou le fou, il est nécessaire de créer une fonction qui indique si le déplacement que l'on souhaite n'implique pas de traverser une pièce, auquel cas ce déplacement est impossible.

In [7]:
# TRAVERSE TOUR

In [8]:
# DEPLACEMENT TOUR

In [9]:
# TRAVERSE FOU

In [10]:
# DEPLACEMENT FOU

Dès lors, nous avons pu combiner ces deux fonctions pour programmer les déplacements de la dame.

In [26]:
def deplacement_dame(p, a, b, plateau, morts) :                        
    (X, Y) = p[0]
    if a == X or b == Y :                               # déplacement selon une ligne/colonne
        deplacement_tour(p, a, b, plateau, morts)                       
    else:                                               # déplacement sur une diagonale
        deplacement_fou(p, a, b, plateau, morts)

Ensuite, nous avons codé les règles de déplacement du cavalier.

In [27]:
# DEPLACEMENT CAVALIER

Enfin, il ne restait qu'à coder les déplacements du roi, qui sont assez basiques par rapport à ceux des autres pièces.

In [28]:
# DEPLACEMENT ROI

Renvoyer une ValueError pour chaque fonction dès qu'une erreur a été détectée permettra d'utiliser ces fonctions dans des fonctions annexes. A la différence d'un print et grâce à la structure try/except, cette fonction ne renverra rien lorsque par exemple une tentative de déplacement a échoué.

A partir de toutes ces fonctions spécifiques à chaque type de pièce, nous définissons la fonction déplacement. Cette fonction permet de déplacer une pièce en utilisant les fonctions précédentes

In [29]:
def deplacement(p,a,b, plateau, morts) :                         
    name = p[1]
    if name == "pion":
        deplacement_pion(p, a, b, plateau, morts)
    elif name == "fou":
        deplacement_fou(p, a, b, plateau, morts)
    elif name == "cavalier":
        deplacement_cavalier(p, a, b, plateau, morts)
    elif name == "tour":
        deplacement_tour(p, a, b, plateau, morts)
    elif name == "dame":
        deplacement_dame(p, a, b, plateau, morts)
    else:
        deplacement_roi(p, a, b, plateau, morts)

## Autres règles du jeu d'échecs

Maintenant que nous avons établi des règles sur les déplacements de chaque type de pièce, il reste encore à pouvoir effectuer un roque (petit ou grand), à définir le cas d'échec ainsi que le cas d'échec et mat.

## Petit et grand roques

En ce qui concerne le roque, il est nécessaire de vérifier si le roi que l'on veut faire roquer a déjà été mis en échec, auquel cas le roque est impossible. Cet indicateur est caractérisé par nb_morts, un dictionnaire à deux entrées, "blanc" et "noir".

In [15]:
# ROQUE

## Caractérisation de l'échec

Intéressons-nous désormais à la règle de l'échec. Pour avoir un échec, il faut que le roi puisse être éliminé par une pièce adverse au tour suivant. Tout d'abord, nous avons besoin d'une fonction qui donne la liste des pièces blanches et des pièces noires.

In [16]:
def find_color(plateau):
    blancs, noirs = [], []
    for i in range(0,8):
        for j in range(0,8):
            if plateau[i,j][3] == 0 :
                pass
            else:
                piece = plateau[i,j]
                if piece[2] == "noir":
                    noirs.append(piece)
                else :
                    blancs.append(piece)
    return noirs, blancs  

Ensuite, il faut une fonction qui détermine pour chaque couleur toutes les cases qui peuvent être atteintes par cette couleur. Comme indiqué précédemment, on teste les fonctions de déplacement sans risquer de déclencher une erreur grâce à la structure try/except.

In [17]:
def liste_deplacements_possibles(color, plateau, morts) :
    liste_deplacements = []
    copie_plateau = copy.copy(plateau)
    for piece in color :
        for a in range(0,8) :
            for b in range(0,8) :
                try :
                    deplacement(piece,a,b, copie_plateau, morts)
                    liste_deplacements.append([a,b])
                except :
                    pass
    return liste_deplacements

Enfin, on détermine si les coordonnées de chaque roi peuvent être atteintes par une pièce adverse. Si l'une des couleurs est mise en échec, test_echec renverra True et color, avec color la liste des pièces de la couleur mise en échec.

In [18]:
def test_echec(plateau, morts, nb_echecs):
    noirs, blancs = find_color(plateau)
    deplacements_noirs = liste_deplacements_possibles(noirs, plateau, morts)
    deplacements_blancs = liste_deplacements_possibles(blancs, plateau, morts)
    if RB[0] in deplacements_noirs :
        nb_echecs["blanc"] += 1
        print("Le roi blanc est en échec")
        return True, blancs, nb_echecs
    elif RN[0] in deplacements_blancs :
        nb_echecs["noir"] += 1
        print("Le roi noir est en échec")
        return True, noirs, nb_echecs
    return False

Pour en finir avec les règles, il a fallu caractériser l'échec et mat. Pour cela, il faut tester s'il existe un déplacement possible de la couleur mise en échec pour ne plus être en échec. Sinon, il y a échec et mat. On aura donc besoin de la fonction liste_deplacements_piece qui donne la liste des déplacements pour chaque pièce. Puis on teste sur une copie du plateau si l'un des déplacements possibles de l'une des pièces permet de sortir de l'échec.

In [19]:
def liste_deplacements_piece(piece, plateau, morts):
    liste_deplacements = []
    copie_plateau = copy.copy(plateau)
    for a in range(0,8) :
        for b in range(0,8) :
            try :
                deplacement(piece,a,b,copie_plateau, morts)
                liste_deplacements.append([a,b])
            except :
                pass
    return liste_deplacements

In [21]:
def echec_et_mat(plateau, morts, nb_echecs):
    if test_echec(plateau, morts) == False :
        pass
    else :
        test, couleur, nb_echecs = test_echec(plateau, morts, nb_echecs)                         # couleur = couleur mise en échec
        for piece in couleur :                                             # on teste toutes les pièces de la couleur mise en échec
            liste = liste_deplacements_piece(piece, plateau, morts)        # liste des déplacements possibles pour chaque pièce
            for couple in liste :
                a = couple[0]
                b = couple[1]
                copie_nb_echecs = copy.copy(nb_echecs)
                copie_plateau = copy.copy(plateau)                         # on effectue une copie du plateau pour simuler les déplacements
                deplacement(piece, a, b, copie_plateau, morts)
                if test_echec(copie_plateau, morts, copie_nb_echecs) == False :             # on teste si après un déplacement possible, le joueur peut sortir de l'échec
                    return False   # il n'y a pas échec et mat
                return True        # il y a pas échec et mat            

Pour conclure sur l'implémentation des règles, il a fallu créer une fonction qui indique si la partie est finie et auquel cas quelle est la couleur gagnante.

In [22]:
def fin_de_partie(plateau) :                      
    if echec_et_mat(plateau) == False :
        return None
    else :                                                    # il y a échec et mat 
        test, couleur = test_echec(plateau)                   # la couleur perdante est celle mise en échec
        if couleur == "blancs" :
            print("Victoire des noirs !!")
        else :
            print("Victoire des blancs !!")

# Mise en place de l'interface graphique
## Affichage des pièces et du plateau

Comme indiqué précédemment, nous avons crée les représentations graphiques de chaque pièce via le module tkinter. Les pièces se trouvent dans le dossier joint. Il suffit de remplacer leurs chemins pour pouvoir les afficher. 

In [23]:
plateau_visuel = tkinter.Tk()

case_vide = tkinter.PhotoImage(file='C:/Users/yohan/Desktop/pieces_echecs_info/case_vide.gif')
Dame_noire = tkinter.PhotoImage(file='C:/Users/yohan/Desktop/pieces_echecs_info/Dame_noire.gif')
Roi_noir = tkinter.PhotoImage(file='C:/Users/yohan/Desktop/pieces_echecs_info/Roi_noir.gif')
Tour_noire = tkinter.PhotoImage(file='C:/Users/yohan/Desktop/pieces_echecs_info/Tour_noire.gif')
Fou_noir = tkinter.PhotoImage(file='C:/Users/yohan/Desktop/pieces_echecs_info/Fou_noir.gif')
Cavalier_noir = tkinter.PhotoImage(file='C:/Users/yohan/Desktop/pieces_echecs_info/Cavalier_noir.gif')
Pion_noir = tkinter.PhotoImage(file='C:/Users/yohan/Desktop/pieces_echecs_info/Pion_noir.gif')
Dame_blanche = tkinter.PhotoImage(file='C:/Users/yohan/Desktop/pieces_echecs_info/Dame_blanche.gif')
Roi_blanc = tkinter.PhotoImage(file='C:/Users/yohan/Desktop/pieces_echecs_info/Roi_blanc.gif')
Tour_blanche = tkinter.PhotoImage(file='C:/Users/yohan/Desktop/pieces_echecs_info/Tour_blanche.gif')
Fou_blanc = tkinter.PhotoImage(file='C:/Users/yohan/Desktop/pieces_echecs_info/Fou_blanc.gif')
Cavalier_blanc = tkinter.PhotoImage(file='C:/Users/yohan/Desktop/pieces_echecs_info/Cavalier_blanc.gif')
Pion_blanc = tkinter.PhotoImage(file='C:/Users/yohan/Desktop/pieces_echecs_info/Pion_blanc.gif')

TclError: couldn't open "C:/Users/yohan/Desktop/pieces_echecs_info/case_vide.gif": no such file or directory

Puis nous avons crée les pièces et le plateau selon le modèle donné : [(X,Y), type, couleur, numero, image] .

In [ ]:
PN1 =[[1,0],"pion","noir",1, Pion_noir]
PN2 =[[1,1],"pion","noir",2, Pion_noir]
PN3 =[[1,2],"pion","noir",3, Pion_noir]
PN4 =[[1,3],"pion","noir",4, Pion_noir]
PN5 =[[1,4],"pion","noir",5, Pion_noir]
PN6 =[[1,5],"pion","noir",6, Pion_noir]
PN7 =[[1,6],"pion","noir",7, Pion_noir]
PN8 =[[1,7],"pion","noir",8, Pion_noir]

TN1 = [[0,0],"tour","noir",1, Tour_noire]
CN1 = [[0,1],"cavalier","noir",1, Cavalier_noir]
FN1 = [[0,2],"fou","noir",1, Fou_noir]        #fou qui n'ira que sur les cases blanches
DN = [[0,3],"dame","noir",1, Dame_noire]
RN = [[0,4],"roi","noir",1, Roi_noir]
FN2 = [[0,5],"fou","noir",2, Fou_noir]        #fou qui n'ira que sur les cases noires
CN2 = [[0,6],"cavalier","noir",2, Cavalier_noir]
TN2 = [[0,7],"tour","noir",2, Tour_noire]

PB1 =[[6,0],"pion","blanc",1, Pion_blanc]
PB2 =[[6,0],"pion","blanc",2, Pion_blanc]
PB3 =[[6,0],"pion","blanc",3, Pion_blanc]
PB4 =[[6,0],"pion","blanc",4, Pion_blanc]
PB5 =[[6,0],"pion","blanc",5, Pion_blanc]
PB6 =[[6,0],"pion","blanc",6, Pion_blanc]
PB7 =[[6,0],"pion","blanc",7, Pion_blanc]
PB8 =[[6,0],"pion","blanc",8, Pion_blanc]

TB2 = [[7,0],"tour","blanc",2, Tour_blanche]
CB2 = [[7,1],"cavalier","blanc",2, Cavalier_blanc]
FB2 = [[7,2],"fou","blanc",2, Fou_blanc]       #fou qui n'ira que sur les cases noires
DB = [[7,3],"dame","blanc",1, Dame_blanche]
RB = [[7,4],"roi","blanc",1, Roi_blanc]
FB1 = [[7,5],"fou","blanc",1, Fou_blanc]       #fou qui n'ira que sur les cases blanches
CB1 = [[7,6],"cavalier","blanc",1, Cavalier_blanc]
TB1 = [[7,7],"tour","blanc",1, Tour_blanche]

CV = [[0,0],"case_vide","",0,case_vide]

plateau = numpy.array([[TN1,CN1,FN1,DN,RN,FN2,CN2,TN2],[PN1,PN2,PN3,PN4,PN5,PN6,PN7,PN8],[CV,CV,CV,CV,CV,CV,CV,CV],[CV,CV,CV,CV,CV,CV,CV,CV],[CV,CV,CV,CV,CV,CV,CV,CV],[CV,CV,CV,CV,CV,CV,CV,CV],[PB1,PB2,PB3,PB4,PB5,PB6,PB7,PB8],[TB1,CB1,FB2,DB,RB,FB1,CB2,TB2]])